<a href="https://colab.research.google.com/github/john-d-noble/callcenter/blob/main/callcenter%20w/%20call%20volatility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas scikit-learn xgboost statsmodels tensorflow

In [2]:
# === DO NOT MANUALLY SELECT THIS TEXT. USE THE COPY BUTTON. ===

import pandas as pd
import numpy as np
import warnings

# Import ML and Stats Libraries
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

def run_forecasting_analysis():
    """
    This function encapsulates the entire analysis process.
    """
    # --- 1. Data Loading and Preparation ---
    print("Step 1: Loading and preparing data...")

    try:
        # Find the correct header row number first
        header_row_number = 0
        with open('/content/final_simulated_data_with_volatility.csv', 'r') as f:
            for i, line in enumerate(f):
                if line.strip().startswith('date'):
                    header_row_number = i
                    break

        # Load the data using the dynamically found header row
        df = pd.read_csv(
            '/content/final_simulated_data_with_volatility.csv',
            sep='\s+',
            skiprows=header_row_number,
            skipfooter=1,
            engine='python'
        )
    except FileNotFoundError:
        print("\nERROR: 'final_simulated_data_with_volatility.csv' not found.")
        print("Please make sure the data file is in the correct location.\n")
        return

    df.columns = ['date', 'btc_price', 'eth_price', 'sol_price', 'vix_index', 'call_volume']

    # Clean the data
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.drop(['eth_price', 'sol_price'], axis=1, inplace=True)
    df.sort_index(inplace=True)
    print("Data successfully loaded and cleaned.")

    # --- 2. Feature Engineering ---
    print("Step 2: Engineering features...")
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['week_of_year'] = df.index.isocalendar().week.astype(int)
    df['year'] = df.index.year
    df['call_volume_lag1'] = df['call_volume'].shift(1)
    df['call_volume_lag7'] = df['call_volume'].shift(7)
    df.dropna(inplace=True)

    # --- 3. Data Splitting ---
    print("Step 3: Splitting data into training and testing sets...")
    test_size = 90
    train_df = df[:-test_size]
    test_df = df[-test_size:]

    X_train = train_df.drop('call_volume', axis=1)
    y_train = train_df['call_volume']
    X_test = test_df.drop('call_volume', axis=1)
    y_test = test_df['call_volume']

    results = {}

    # --- 4. Model Training and Evaluation ---
    print("\n--- Running Models & Displaying Inline Results ---")
    header = f"{'Model':<20} | {'MAE':>8} | {'RMSE':>8} | {'MAPE (%)':>10} | {'Improvement (%)':>18}"
    print(header)
    print("-" * len(header))

    # Model 1: Baseline (Naive Forecast)
    baseline_preds = test_df['call_volume_lag7']
    baseline_mae = mean_absolute_error(y_test, baseline_preds)
    baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_preds))
    baseline_mape = np.mean(np.abs((y_test - baseline_preds) / y_test)) * 100
    results['Baseline (Naive)'] = {'MAE': baseline_mae, 'RMSE': baseline_rmse, 'MAPE (%)': baseline_mape}
    print(f"{'Baseline (Naive)':<20} | {baseline_mae:>8.2f} | {baseline_rmse:>8.2f} | {baseline_mape:>10.2f} | {'N/A':>18}")

    # Model 2: Linear Regression
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_preds = lr_model.predict(X_test)
    lr_mae = mean_absolute_error(y_test, lr_preds)
    lr_rmse = np.sqrt(mean_squared_error(y_test, lr_preds))
    lr_mape = np.mean(np.abs((y_test - lr_preds) / y_test)) * 100
    results['Linear Regression'] = {'MAE': lr_mae, 'RMSE': lr_rmse, 'MAPE (%)': lr_mape}
    improvement = 100 * (baseline_mape - lr_mape) / baseline_mape
    print(f"{'Linear Regression':<20} | {lr_mae:>8.2f} | {lr_rmse:>8.2f} | {lr_mape:>10.2f} | {improvement:>18.2f}")

    # Model 3: ARIMA
    arima_model = ARIMA(endog=y_train, exog=X_train, order=(5, 1, 0))
    arima_results = arima_model.fit()
    arima_preds = arima_results.forecast(steps=len(X_test), exog=X_test)
    arima_mae = mean_absolute_error(y_test, arima_preds)
    arima_rmse = np.sqrt(mean_squared_error(y_test, arima_preds))
    arima_mape = np.mean(np.abs((y_test - arima_preds) / y_test)) * 100
    results['ARIMA'] = {'MAE': arima_mae, 'RMSE': arima_rmse, 'MAPE (%)': arima_mape}
    improvement = 100 * (baseline_mape - arima_mape) / baseline_mape
    print(f"{'ARIMA':<20} | {arima_mae:>8.2f} | {arima_rmse:>8.2f} | {arima_mape:>10.2f} | {improvement:>18.2f}")

    # Model 4: XGBoost
    xgb_model = xgb.XGBRegressor(
        objective='reg:squarederror', n_estimators=1000, learning_rate=0.01,
        max_depth=5, subsample=0.8, colsample_bytree=0.8, random_state=42,
        early_stopping_rounds=50
    )
    xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    xgb_preds = xgb_model.predict(X_test)
    xgb_mae = mean_absolute_error(y_test, xgb_preds)
    xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_preds))
    xgb_mape = np.mean(np.abs((y_test - xgb_preds) / y_test)) * 100
    results['XGBoost'] = {'MAE': xgb_mae, 'RMSE': xgb_rmse, 'MAPE (%)': xgb_mape}
    improvement = 100 * (baseline_mape - xgb_mape) / baseline_mape
    print(f"{'XGBoost':<20} | {xgb_mae:>8.2f} | {xgb_rmse:>8.2f} | {xgb_mape:>10.2f} | {improvement:>18.2f}")

    # Model 5: LSTM
    print("Running LSTM model (this may take a few minutes)...")
    scaler_features = MinMaxScaler()
    scaler_target = MinMaxScaler()
    X_train_scaled = scaler_features.fit_transform(X_train)
    X_test_scaled = scaler_features.transform(X_test)
    y_train_scaled = scaler_target.fit_transform(y_train.values.reshape(-1, 1))

    def create_sequences(X, y, time_steps=7):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X[i:(i + time_steps)])
            ys.append(y[i + time_steps])
        return np.array(Xs), np.array(ys)

    TIME_STEPS = 7
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, TIME_STEPS)

    lstm_model = Sequential([
        LSTM(50, activation='relu', input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
        Dropout(0.2),
        Dense(1)
    ])
    lstm_model.compile(optimizer='adam', loss='mean_squared_error')
    lstm_model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=32, verbose=0, shuffle=False)

    padded_X_test = np.concatenate([X_train_scaled[-TIME_STEPS:], X_test_scaled])
    X_test_seq, _ = create_sequences(padded_X_test, np.zeros(len(padded_X_test)), TIME_STEPS)

    lstm_preds_scaled = lstm_model.predict(X_test_seq, verbose=0)
    lstm_preds = scaler_target.inverse_transform(lstm_preds_scaled).flatten()

    lstm_mae = mean_absolute_error(y_test, lstm_preds)
    lstm_rmse = np.sqrt(mean_squared_error(y_test, lstm_preds))
    lstm_mape = np.mean(np.abs((y_test - lstm_preds) / y_test)) * 100
    results['LSTM'] = {'MAE': lstm_mae, 'RMSE': lstm_rmse, 'MAPE (%)': lstm_mape}
    improvement = 100 * (baseline_mape - lstm_mape) / baseline_mape
    print(f"{'LSTM':<20} | {lstm_mae:>8.2f} | {lstm_rmse:>8.2f} | {lstm_mape:>10.2f} | {improvement:>18.2f}")

    # --- 5. Generate and Print Final Summary Table ---
    print("\n\n--- Final Summary Table ---")
    results_df = pd.DataFrame.from_dict(results, orient='index')
    results_df = results_df.reset_index().rename(columns={'index': 'Model'})
    results_df['Improvement over Baseline (%)'] = 100 * (baseline_mape - results_df['MAPE (%)']) / baseline_mape
    results_df = results_df.round(2).sort_values(by='MAPE (%)', ascending=True)
    print(results_df.to_string(index=False))
    print("\n--- Analysis Complete ---")


# --- Execute the entire analysis ---
if __name__ == "__main__":
    run_forecasting_analysis()

<>:38: SyntaxWarning: invalid escape sequence '\s'
<>:38: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1613536554.py:38: SyntaxWarning: invalid escape sequence '\s'
  sep='\s+',


Step 1: Loading and preparing data...


ValueError: Length mismatch: Expected axis has 1 elements, new values have 6 elements